In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import math

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (11, 7)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna 

# Feel free to add all the libraries you need
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from geopy.distance import geodesic

In [100]:
df1 = pd.read_csv('data/labelencoded.csv', index_col=0)

In [101]:
df = df1[df1['target']!=0]

In [102]:
# define features and target
features = df.columns.tolist()
features.remove('target')
X = df[features]
y = df['target']

# LETS DO FEATURE ENGINEERING.....gonna square some shit 
# generate interaction features to the 3rd degree
p = PolynomialFeatures(interaction_only=False, include_bias=False, degree=1)
X_poly = p.fit_transform(X)
p.n_output_features_

27

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state=12, test_size=0.5)

In [108]:
## feature engineering -- create function to pull out coefficients/features
## add a gradient descent
def lasso_optimization(x123_train):
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[], 'alpha':[], 'coeff_used':[], 'coefficients':[], 'intercept':[]}

    # initialize and train model with (default value) alpha = 0.5
    # for i in [0.1, 1, 10,100,1000, 10000, 33000,66000, 100000, 330000,660000,1000000, 3300000,6600000,10000000,33000000,66000000,100000000]:
    for i in [1, 10, 100, 1000]:

        model = Lasso(alpha=i, max_iter=int(1000))
        model.fit(x123_train,y_train)
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        ## use cross validation for 
        score = cross_validate(model, x123_train, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        
        ## pull out the scores
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        coeff_used = np.sum(model.coef_!=0)
        coefficients = model.coef_
        intercept = model.intercept_

        dicKeys = ['mean_MAPE','mean_r2','mean_MSE','std_MAPE','std_r2','std_MSE','score','alpha','coeff_used', 'coefficients', 'intercept'] 
        dicValues = [mean_MAPE, mean_r2, mean_MSE, std_MAPE, std_r2, std_MSE, len(score), i, coeff_used, coefficients, intercept]
        for (keyi, vali) in zip(dicKeys, dicValues):
            eval_metrics_dictionary[keyi].append(vali)
        #eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        #eval_metrics_dictionary['mean_r2'].append(mean_r2)
        #eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        #eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        #eval_metrics_dictionary['std_r2'].append(std_r2)
        #eval_metrics_dictionary['std_MSE'].append(std_MSE)
        #eval_metrics_dictionary['score'].append(len(score))
        #eval_metrics_dictionary['alpha'].append(i)
        #eval_metrics_dictionary['coeff_used'].append(coeff_used)

    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue
#return df_framework.sort_values('combined_score')

In [109]:
hi = lasso_optimization(X_train)

c:\Users\erick\gitrepositories\flight_delays_ML\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.965e+07, tolerance: 4.831e+04
  model = cd_fast.enet_coordinate_descent(


In [110]:
hi

,mean_r2,mean_MAPE,mean_MSE,std_r2,std_MAPE,std_MSE,score,alpha,coeff_used,coefficients,intercept,combined_score
0,0.023187,2.976948,19231.288366,0.001005,0.068948,1799.078446,5,1,21,"[0.0014450860490376252, -0.056552885094557755,...",-29103.564121,55923.07879


In [111]:
# take list of coefficients - if more than zero 
# keep the name of the poly feature
# keep the non-zero poly features values
featurenames = list(p.get_feature_names_out(X.columns))
##fix the zero
coefficient_list = list(hi['coefficients'][0]) 

### creature a dictionary of features and coefficients and then turn into dataframe
feature_coefficient_dict = {'feature':[], 'coefficient':[]}
for (feaI, coeI) in zip(featurenames, coefficient_list): 
    if coeI != 0: 
        feature_coefficient_dict['feature'].append(feaI)
        feature_coefficient_dict['coefficient'].append(coeI)
        
pd.DataFrame(feature_coefficient_dict)
feature_selection = feature_coefficient_dict['feature']
#### with the coefficients create a new np.array to 
Xtrain1 = np.array(pd.DataFrame(X_train, columns= p.get_feature_names_out(X.columns))[feature_selection])

In [112]:
hi = lasso_optimization(Xtrain1)

In [113]:
hi

,mean_r2,mean_MAPE,mean_MSE,std_r2,std_MAPE,std_MSE,score,alpha,coeff_used,coefficients,intercept,combined_score
0,0.023217,2.97702,19230.768267,0.000977,0.069052,1799.697369,5,1,21,"[0.0014450859119771776, -0.057313962698797224,...",-29103.570323,55921.229329


In [114]:
# take list of coefficients - if more than zero 
# keep the name of the poly feature
# keep the non-zero poly features values
featurenames = list(p.get_feature_names_out(X.columns))
##fix the zero
coefficient_list = list(hi['coefficients'][0]) 

### creature a dictionary of features and coefficients and then turn into dataframe
feature_coefficient_dict = {'feature':[], 'coefficient':[]}
for (feaI, coeI) in zip(featurenames, coefficient_list): 
    if coeI != 0: 
        feature_coefficient_dict['feature'].append(feaI)
        feature_coefficient_dict['coefficient'].append(coeI)
        
pd.DataFrame(feature_coefficient_dict)
feature_selection = feature_coefficient_dict['feature']
#### with the coefficients create a new np.array to 
ridge_Xtrain = np.array(pd.DataFrame(X_train, columns= p.get_feature_names_out(X.columns))[feature_selection])

In [115]:
pd.DataFrame(feature_coefficient_dict)


,feature,coefficient
0,datop,1.445086e-03
1,depstn,-5.731396e-02
2,arrstn,-5.951240e-02
3,status,-2.210036e-02
4,ac,1.118436e+00
5,year,9.840891e-02
6,month,-2.077935e-02
7,day,2.994251e-01
8,dep_iata,8.585264e-03
9,dep_country,-8.176810e-01


In [116]:
ridge_Xtrain

array([[ 2.0170829e+07,  1.2300000e+02,  4.8000000e+01, ...,
         2.0000000e+00,  1.4000000e+01,  3.4200000e+02],
       [ 2.0160812e+07,  4.8000000e+01,  1.1900000e+02, ...,
         2.0000000e+00,  1.4000000e+01, -3.4200000e+02],
       [ 2.0160622e+07,  5.9000000e+01,  1.1900000e+02, ...,
         2.0000000e+00,  1.4000000e+01, -2.6000000e+01],
       ...,
       [ 2.0170716e+07,  9.6000000e+01,  1.1900000e+02, ...,
         2.0000000e+00,  1.4000000e+01, -2.6900000e+02],
       [ 2.0171016e+07,  5.8000000e+01,  1.1900000e+02, ...,
         0.0000000e+00,  1.4000000e+01, -3.0300000e+02],
       [ 2.0161010e+07,  1.1700000e+02,  1.1900000e+02, ...,
         0.0000000e+00,  1.4000000e+01, -4.7700000e+02]])

In [122]:
def ridge_optimization(number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[]}
    ridge_opt_dictionary = { 'alpha': [], 'intercept' : [], 'tol': [], 'solver':[], 'no_of_coefficients':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        alpha = trial.suggest_float("alpha", 0, 100)
        intercept = trial.suggest_categorical("fit_intercept", [True, False])
        tol = trial.suggest_float("tol", 0.001, 1, log=True)
        solver = trial.suggest_categorical("solver", ["auto", "svd","cholesky", "lsqr", "saga", "sag"])

        ## MODEL SELECTION
        ## Create Model
        model = Ridge(alpha=alpha,fit_intercept=intercept,tol=tol,solver=solver)
        ## Fit Model
        model.fit(ridge_Xtrain, y_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, X_train, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        ridge_opt_dictionary['alpha'].append(alpha)
        ridge_opt_dictionary['intercept'].append(intercept)
        ridge_opt_dictionary['tol'].append(tol)
        ridge_opt_dictionary['solver'].append(solver)
        ridge_opt_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_r2

    study2 = optuna.create_study(study_name="RidgeRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue

In [123]:
ridge_optimization(200)

[I 2023-08-31 22:50:32,350] A new study created in memory with name: RidgeRegression Optimization
[W 2023-08-31 22:50:55,481] Trial 0 failed with parameters: {'alpha': 75.61895757061254, 'fit_intercept': False, 'tol': 0.00102216412755496, 'solver': 'sag'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\erick\gitrepositories\flight_delays_ML\.venv\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\erick\AppData\Local\Temp\ipykernel_24896\951272127.py", line 37, in objective
    score = cross_validate(model, X_train, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\erick\gitrepositories\flight_delays_ML\.venv\Lib\site-packages\sklea

KeyboardInterrupt: 